In [1]:
import pandas as pd
from datetime import datetime
import seaborn as sns
from sklearn.model_selection import train_test_split

train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [2]:
X = train.drop(columns = "target")
y = train["target"]

In [3]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [4]:
X_train['target'] = y_train
X_val['target'] = y_val

In [5]:
train = X_train.copy()
val = X_val.copy()

In [6]:
train = train.dropna()
val = val.dropna()

In [7]:
#Keep average cost min
averagecostmin = val['average cost min']
idtest = test["id"]

### Initial Exploration

In [8]:
train.describe()

,Age,L_O_S,Dropped_Calls,Peak_calls_Sum,Peak_mins_Sum,OffPeak_calls_Sum,OffPeak_mins_Sum,Weekend_calls_Sum,Weekend_mins_Sum,International_mins_Sum,...,call_cost_per_min,actual call cost,Total_call_cost,Total_Cost,average cost min,Peak ratio,OffPeak ratio,Weekend ratio,Nat-InterNat Ratio,target
count,4032.000000,4032.000000,4032.000000,4032.000000,4032.000000,4032.000000,4032.000000,4032.000000,4032.000000,4032.000000,...,4032.000000,4032.000000,4032.000000,4032.000000,4032.000000,4032.000000,4032.000000,4032.000000,4032.000000,4032.000000
mean,31.418403,33.692675,2.673115,239.123016,709.319559,104.113839,312.170469,16.334821,50.117233,169.677704,...,10.081397,19.148192,73.555944,186.897819,0.168059,0.610169,0.328068,0.061763,0.162448,0.146329
std,12.778218,14.006476,3.419745,239.289112,503.745589,97.255574,199.059794,16.296709,36.080006,141.764351,...,2.108023,27.992736,59.989152,82.037001,0.077313,0.223896,0.209820,0.063645,0.105589,0.353480
min,12.000000,9.633333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.034634,...,2.000000,0.000000,0.010390,60.000842,0.048998,0.000000,0.000000,0.000000,0.000030,0.000000
25%,22.000000,21.291667,0.000000,59.000000,307.800001,28.000000,150.000000,4.000000,22.800000,64.538731,...,8.573250,0.000000,32.168235,123.926851,0.134406,0.466570,0.165197,0.020977,0.079269,0.000000
50%,29.000000,33.650000,1.000000,161.000000,615.000000,73.000000,291.150000,11.000000,44.399999,132.301181,...,9.518597,7.547389,57.875946,173.404650,0.154501,0.654919,0.287928,0.043940,0.157865,0.000000
75%,39.000000,45.933333,2.000000,343.000000,1014.900000,156.000000,446.475000,24.000000,72.000000,236.883563,...,11.696253,28.328732,99.290441,232.132750,0.179244,0.782404,0.458625,0.081047,0.244300,0.000000
max,80.000000,58.200000,15.000000,1626.000000,2901.600001,560.000000,1091.099999,106.000000,205.000000,935.947864,...,21.734694,184.892166,437.063835,587.063835,1.357564,1.000000,1.000000,0.731884,2.601223,1.000000


In [9]:
train.describe(include = 'object')

,Gender,Connect_Date,tariff,Handset,Usage_Band,Tariff_OK,high Dropped calls,No Usage,id
count,4032,4032,4032,4032,4032,4032,4032,4032,4032
unique,2,1310,5,11,5,4,2,1,4032
top,F,11/07/99,CAT 200,S50,Med,OK,F,F,K277140
freq,2040,11,1802,944,2232,4003,3930,4032,1


Drop ID, drop Connect_Date (too many unique values, do not provide information on response)

In [10]:
train = train.drop(columns = ["id", "Connect_Date"])

In [11]:
val = val.drop(columns = ["id", "Connect_Date"])
test = test.drop(columns = ["id", "Connect_Date"])

#### Dropping numerical variables that are useless

In [12]:
correlation_matrix = train[train.select_dtypes(include=['number']).columns.tolist()].corr()

In [13]:
corrtarget = correlation_matrix['target']

In [14]:
# Make plot for this

In [15]:
high_correlation_pairs = []
for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        if abs(correlation_matrix.iloc[i, j]) > 0.8:
            high_correlation_pairs.append((correlation_matrix.columns[i], correlation_matrix.columns[j]))


In [16]:
high_correlation_pairs

[('Peak_calls_Sum', 'National_calls'),
 ('Peak_mins_Sum', 'National mins'),
 ('Peak_mins_Sum', 'All_calls_mins'),
 ('Peak_mins_Sum', 'Total_Cost'),
 ('International_mins_Sum', 'Total_call_cost'),
 ('International_mins_Sum', 'Total_Cost'),
 ('Nat_call_cost_Sum', 'Mins_charge'),
 ('Nat_call_cost_Sum', 'actual call cost'),
 ('National mins', 'All_calls_mins'),
 ('National mins', 'Total_Cost'),
 ('All_calls_mins', 'Total_call_cost'),
 ('All_calls_mins', 'Total_Cost'),
 ('Mins_charge', 'actual call cost'),
 ('Total_call_cost', 'Total_Cost'),
 ('Peak ratio', 'OffPeak ratio')]

In [17]:
from collections import defaultdict

def build_adjacency_list(pairs):
    adjacency_list = defaultdict(list)
    for u, v in pairs:
        adjacency_list[u].append(v)
        adjacency_list[v].append(u)
    return adjacency_list

# Function to perform depth-first search (DFS) traversal to find connected components
def dfs(node, adjacency_list, visited, component):
    visited.add(node)
    component.append(node)
    for neighbor in adjacency_list[node]:
        if neighbor not in visited:
            dfs(neighbor, adjacency_list, visited, component)

# Function to find connected components in the graph
def find_connected_components(pairs):
    adjacency_list = build_adjacency_list(pairs)
    visited = set()
    connected_components = []
    for node in adjacency_list:
        if node not in visited:
            component = []
            dfs(node, adjacency_list, visited, component)
            connected_components.append(component)
    return connected_components

# Find connected groups
connected_groups = find_connected_components(high_correlation_pairs)

In [18]:
def get_variables_to_drop(group, corrtarget):
    max_cor = 0
    var_max = ''
    for var in group:
        if abs(corrtarget[var]) > max_cor:
            var_max = var
    group.remove(var_max)

    return group

In [19]:
variables_to_drop = [get_variables_to_drop(group, corrtarget) for group in connected_groups]

In [20]:
drop_vars = [item for sublist in variables_to_drop for item in sublist]

In [21]:
train = train.drop(columns = drop_vars)

In [22]:
val = val.drop(columns = drop_vars)

In [23]:
test = test.drop(columns = drop_vars)

#### Deleting categorical variables that are useless

In [24]:
for variable in list(train.describe(include = 'object').columns):
    print(train.groupby('target')[variable].value_counts())

target  Gender
0       F         1760
        M         1682
1       M          310
        F          280
Name: count, dtype: int64
target  tariff  
0       CAT 200     1533
        CAT 100      736
        Play 100     502
        Play 300     471
        CAT 50       200
1       CAT 200      269
        Play 100     126
        CAT 100      110
        Play 300      47
        CAT 50        38
Name: count, dtype: int64
target  Handset
0       S50        825
        BS110      598
        S80        574
        WC95       526
        ASAD170    514
        BS210      220
        CAS60       90
        ASAD90      42
        CAS30       37
        SOP20       10
        SOP10        6
1       ASAD90     174
        S50        119
        CAS30      106
        BS110       95
        SOP10       28
        SOP20       24
        S80         20
        ASAD170     10
        BS210        8
        WC95         6
Name: count, dtype: int64
target  Usage_Band
0       Med           1973
   

In [25]:
#Make plots for this

No Usage, UsageBand, Tariff seem totally useless, they are dropped for now
Most useful inferences: High Dropped Calls seems a good indicator for whether someone will churn or not, ratio of Trues for churners is much higher. Tariff_Ok's High Cat 100 for churners is much higher than for non churners, would be good to add numerical encoding. Same for handset, numerical encoding would be nice. 

In [26]:
train = train.drop(columns = ["No Usage", "Usage_Band", "tariff"])
test = test.drop(columns = ["No Usage", "Usage_Band", "tariff"])
val = val.drop(columns = ["No Usage", "Usage_Band", "tariff"])

#### Handling Missing Values

In [27]:
train.isna().sum()

Gender                 0
Age                    0
L_O_S                  0
Dropped_Calls          0
Handset                0
OffPeak_calls_Sum      0
OffPeak_mins_Sum       0
Weekend_calls_Sum      0
Weekend_mins_Sum       0
AvePeak                0
AveOffPeak             0
AveWeekend             0
National_calls         0
AveNational            0
Dropped_calls_ratio    0
call_cost_per_min      0
actual call cost       0
Total_Cost             0
Tariff_OK              0
average cost min       0
OffPeak ratio          0
Weekend ratio          0
Nat-InterNat Ratio     0
high Dropped calls     0
target                 0
dtype: int64

Almost no missing values, these could be imputed

In [28]:
rows_with_missing_values = train[train.isnull().any(axis=1)]


In [29]:
rows_with_missing_values[["call_cost_per_min", "Dropped_calls_ratio", "target"]]

,call_cost_per_min,Dropped_calls_ratio,target


Missing values are all in the same 4 rows. Since they're only 4 observations out of 5000, they're eliminated from the data

Outliers are not dealth with (they are useful in our detection, the higher the costs the bigger the probability of churning?)

### Transformation of Variables

##### Numerical Variables

In [30]:
test['target'] = 0

In [31]:
test

,Gender,Age,L_O_S,Dropped_Calls,Handset,OffPeak_calls_Sum,OffPeak_mins_Sum,Weekend_calls_Sum,Weekend_mins_Sum,AvePeak,...,call_cost_per_min,actual call cost,Total_Cost,Tariff_OK,average cost min,OffPeak ratio,Weekend ratio,Nat-InterNat Ratio,high Dropped calls,target
0,F,20.0,55.582251,3.0,WC95,110.0,189.982227,8.0,5.017969,9.598648,...,8.933287,9.409262,167.242853,OK,0.116924,0.173365,0.009823,0.020349,F,0
1,F,54.0,32.067010,0.0,BS110,93.0,551.817423,8.0,17.593417,33.547208,...,11.931697,2.739423,136.404621,OK,0.120457,0.532898,0.022391,0.124480,F,0
2,F,28.0,26.621954,0.0,ASAD170,90.0,152.731006,10.0,23.907633,2.945673,...,9.560903,95.406158,397.739114,OK,0.149435,0.034434,0.006660,0.225763,F,0
3,F,26.0,27.652735,5.0,WC95,126.0,358.760706,0.0,3.024655,11.984087,...,8.398145,14.464628,196.789012,OK,0.137372,0.263256,0.005870,0.108872,F,0
4,F,34.0,25.479355,0.0,WC95,73.0,141.180280,11.0,47.034972,8.875573,...,12.766109,34.757126,160.450386,OK,0.180704,0.166995,0.056390,0.090896,F,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677,F,32.0,40.097166,0.0,BS110,8.0,42.952890,45.0,105.774031,0.802079,...,9.588439,35.396282,208.523608,OK,0.122342,0.012611,0.070297,0.035339,F,0
1678,M,19.0,22.594431,0.0,BS110,-1.0,73.643988,2.0,28.751955,-2.663694,...,13.794174,30.518649,147.689743,OK,0.163502,0.052157,0.041489,0.079988,F,0
1679,F,34.0,55.339646,0.0,S50,21.0,278.414345,2.0,34.029118,8.078580,...,9.031387,96.373049,302.935896,OK,0.127485,0.106429,0.011459,0.098248,F,0
1680,M,21.0,42.830741,2.0,BS110,76.0,196.742943,-1.0,18.472483,-3.877556,...,11.898371,-0.223523,159.088917,OK,0.205830,0.358159,0.017720,0.256554,F,0


In [32]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
import pandas as pd

# List of numerical variable names
numerical_features = train.select_dtypes(include=['number']).columns.tolist()

# List of categorical variable names (replace with actual categorical variable names)
categorical_features = train.select_dtypes(exclude=['number']).columns.tolist()

# Define the transformers
transformers = [
    ('num', MinMaxScaler(), numerical_features)
]

# Create the column transformer
preprocessor = ColumnTransformer(transformers, remainder='passthrough')

# Apply the column transformer to the data
transformed_train = preprocessor.fit_transform(train)
transformed_val = preprocessor.transform(val)
transformed_test = preprocessor.transform(test)


# Convert the transformed data back to DataFrame
transformed_train = pd.DataFrame(transformed_train, columns=numerical_features + categorical_features)
transformed_test = pd.DataFrame(transformed_test, columns=numerical_features + categorical_features)
transformed_val = pd.DataFrame(transformed_val, columns=numerical_features + categorical_features)



# Only the numerical variables will be scaled to the range [0, 1]

MinMaxScaling (0, 1)

MinMaxScaling (-1, 1)

Normalization + MinMaxScaling

##### Categorical Variables

One-Hot Encoding

In [33]:
for variable in list(train.describe(include = 'object').columns):
    print(train.groupby('target')[variable].value_counts())

target  Gender
0       F         1760
        M         1682
1       M          310
        F          280
Name: count, dtype: int64
target  Handset
0       S50        825
        BS110      598
        S80        574
        WC95       526
        ASAD170    514
        BS210      220
        CAS60       90
        ASAD90      42
        CAS30       37
        SOP20       10
        SOP10        6
1       ASAD90     174
        S50        119
        CAS30      106
        BS110       95
        SOP10       28
        SOP20       24
        S80         20
        ASAD170     10
        BS210        8
        WC95         6
Name: count, dtype: int64
target  Tariff_OK    
0       OK               3435
        High CAT 100        5
        High CAT 50         1
        High Play 100       1
1       OK                568
        High CAT 100       16
        High CAT 50         4
        High Play 100       2
Name: count, dtype: int64
target  high Dropped calls
0       F                  

In [34]:
# Encode binary variables using map
binary_mapping1 = {'M': 1, 'F': 0}
binary_mapping2 = {'T': 1, 'F': 0}

transformed_train['high Dropped calls'] = transformed_train['high Dropped calls'].map(binary_mapping2)
transformed_train['Gender'] = transformed_train['Gender'].map(binary_mapping1)
transformed_test['high Dropped calls'] = transformed_test['high Dropped calls'].map(binary_mapping2)
transformed_test['Gender'] = transformed_test['Gender'].map(binary_mapping1)
transformed_val['high Dropped calls'] = transformed_val['high Dropped calls'].map(binary_mapping2)
transformed_val['Gender'] = transformed_val['Gender'].map(binary_mapping1)


# Filter and encode categorical variables
# For categorical_1, keep only 'High CAT 100', encode the rest as 'Other'
transformed_train['Tariff_OK'] = transformed_train['Tariff_OK'].apply(lambda x: 1 if x == 'High CAT 100' else 0)
transformed_test['Tariff_OK'] = transformed_test['Tariff_OK'].apply(lambda x: 1 if x == 'High CAT 100' else 0)
transformed_val['Tariff_OK'] = transformed_val['Tariff_OK'].apply(lambda x: 1 if x == 'High CAT 100' else 0)

# Encode categorical_2 using one-hot encoding
encoded_train = pd.get_dummies(transformed_train, columns=['Handset'], dtype = int)
encoded_test = pd.get_dummies(transformed_test, columns=['Handset'], dtype = int)
encoded_val = pd.get_dummies(transformed_val, columns=['Handset'], dtype = int)

In [35]:
encoded_train['target'] = encoded_train['target'].astype('category')
encoded_val['target'] = encoded_val['target'].astype('category')

In [36]:
X_train = encoded_train.drop(columns = "target")
y_train = encoded_train["target"]

In [37]:
X_val = encoded_val.drop(columns = "target")
y_val = encoded_val["target"]

In [38]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(max_depth=20, random_state=0)
clf.fit(X_train, y_train)

RandomForestClassifier(max_depth=20, random_state=0)

In [39]:
import numpy as np
from sklearn import metrics
y_pred = clf.predict(X_val)
fpr, tpr, thresholds = metrics.roc_curve(y_val, y_pred)
metrics.auc(fpr, tpr)

0.8125214998280013

In [40]:
def Profit_top_k(df_results, k = 20, var = 'average cost min'):
    sorted_df = df_results.sort_values(by= "predict_proba", ascending = False)
    sum = 0
    for i in range(0, k):
        if df_results['y_true'][i] == df_results['y_pred'][i]:
            sum += df_results[var][i]
    return sum

In [41]:
y_pred_proba = clf.predict_proba(X_val)
y_pred_proba = [sublist[1] for sublist in y_pred_proba]

df_results = pd.DataFrame()
df_results['y_pred'] = y_pred
df_results['y_true'] = y_val
df_results['predict_proba'] = y_pred_proba
df_results['average cost min'] = list(averagecostmin)
Profit_top_k(df_results)

3.2831259999999998

In [42]:
encoded_test = encoded_test.drop(columns = 'target')

In [43]:
from sklearn.impute import SimpleImputer

# Assuming 'data' is your dataset with missing values
# Create an instance of SimpleImputer with strategy='median'
imputer = SimpleImputer(strategy='median')

# Fit the imputer on the data
imputer.fit(X_train)

# Transform the data by replacing missing values with the median
test_clean = imputer.transform(encoded_test)

In [44]:
encoded_test = pd.DataFrame(test_clean, columns= encoded_test.columns)

In [45]:
encoded_test

,Age,L_O_S,Dropped_Calls,OffPeak_calls_Sum,OffPeak_mins_Sum,Weekend_calls_Sum,Weekend_mins_Sum,AvePeak,AveOffPeak,AveWeekend,...,Handset_ASAD90,Handset_BS110,Handset_BS210,Handset_CAS30,Handset_CAS60,Handset_S50,Handset_S80,Handset_SOP10,Handset_SOP20,Handset_WC95
0,0.117647,0.946100,0.200000,0.196429,0.174120,0.075472,0.024478,0.008166,0.000262,0.013685,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.617647,0.461915,0.000000,0.166071,0.505744,0.075472,0.085822,0.028541,0.008096,0.036838,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.235294,0.349800,0.000000,0.160714,0.139979,0.094340,0.116623,0.002506,0.008828,0.015865,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.205882,0.371024,0.333333,0.225000,0.328806,0.000000,0.014754,0.010196,-0.002175,0.064680,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.323529,0.326274,0.000000,0.130357,0.129393,0.103774,0.229439,0.007551,0.009914,0.020227,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677,0.294118,0.627258,0.000000,0.014286,0.039367,0.424528,0.515971,0.000682,0.004807,0.011009,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1678,0.102941,0.266872,0.000000,-0.001786,0.067495,0.018868,0.140253,-0.002266,0.009441,0.065240,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1679,0.323529,0.941105,0.000000,0.037500,0.255168,0.018868,0.165996,0.006873,0.076621,0.078978,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1680,0.132353,0.683543,0.133333,0.135714,0.180316,-0.009434,0.090110,-0.003299,0.008065,0.026010,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [47]:
clf.predict(encoded_test)

array([0., 0., 0., ..., 0., 0., 0.])

In [48]:
y_pred_proba_test = clf.predict_proba(encoded_test)
y_pred_proba_test = [sublist[1] for sublist in y_pred_proba_test]

In [49]:
df_results = pd.DataFrame()
df_results["ID"] = idtest
df_results["PRED"] = y_pred_proba_test

In [52]:
df_results.to_csv("Model_Results1.csv", index = False)

#### Further Investigation of Numerical Variables

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming your data is stored in a DataFrame called df
# and the target variable is named 'target' and numerical variables are named 'num_var1', 'num_var2', ..., 'num_var12'

# Melt the DataFrame to long format for easier plotting
melted_df = df.melt(id_vars='target', value_vars=['num_var1', 'num_var2', 'num_var3', 'num_var4', 'num_var5', 'num_var6', 'num_var7', 'num_var8', 'num_var9', 'num_var10', 'num_var11', 'num_var12'])

# Create boxplot
plt.figure(figsize=(12, 8))
sns.boxplot(x='variable', y='value', hue='target', data=melted_df)
plt.xticks(rotation=45, ha='right')
plt.xlabel('Numerical Variables')
plt.ylabel('Value')
plt.title('Boxplot of Numerical Variables by Target')
plt.legend(title='Target', loc='upper right')
plt.tight_layout()
plt.show()

In [ ]:
#Gradient Boosting
#SVM
#kNN
#RandomForest
#DecisionTree

In [ ]:
#More to explore
#PCA
#Oversampling, undersampling